In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Crohns_Disease/GSE123088'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check for gene expression data availability
# Based on the provided background information, we will assume gene expression data availability
is_gene_available = True

# Analyze the sample characteristics data
sample_characteristics = {
    0: ['cell type: CD4+ T cells'],
    1: [
        'primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 
        'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 
        'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 
        'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS',
        'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 
        'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 
        'primary diagnosis: Breast cancer', 'primary diagnosis: Control'
    ],
    2: [
        'Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 
        'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 
        'diagnosis2: OBESITY'
    ],
    3: [
        'age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 
        'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 
        'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 
        'age: 45', 'age: 65', 'age: 48', 'age: 50', 'age: 24', 'age: 42'
    ],
    4: [float('nan'), 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 'age: 12', 'age: 27']
}

# Identifying keys for each variable
for key, values in sample_characteristics.items():
    if 'primary diagnosis: CROHN_DISEASE' in values:
        trait_row = key
    if any(isinstance(value, str) and 'age:' in value for value in values):
        age_row = key
    if any(isinstance(value, str) and 'Sex: ' in value for value in values):
        gender_row = key

# Define conversion functions
def convert_trait(value):
    if 'primary diagnosis: CROHN_DISEASE' in value:
        return 1
    else:
        return 0

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except Exception:
        return None

def convert_gender(value):
    try:
        gender_val = value.split(': ')[1].strip().lower()
        if gender_val == 'male':
            return 1
        elif gender_val == 'female':
            return 0
        else:
            return None
    except Exception:
        return None

# Save cohort information
save_cohort_info('GSE123088', './preprocessed/Crohns_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical features if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Crohns_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Crohns_Disease/trait_data/GSE123088.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3494884': [0, None, None], 'GSM3494885': [0.0, 63.0, 1.0], 'GSM3494886': [0, None, None], 'GSM3494887': [0, None, None], 'GSM3494888': [0, None, None], 'GSM3494889': [0, None, None], 'GSM3494890': [0.0, 74.0, 1.0], 'GSM3494891': [0, None, None], 'GSM3494892': [0, None, None], 'GSM3494893': [0, None, None], 'GSM3494894': [0, None, None], 'GSM3494895': [0.0, 49.0, 1.0], 'GSM3494896': [0, None, None], 'GSM3494897': [0, None, None], 'GSM3494898': [0, None, None], 'GSM3494899': [0, None, None], 'GSM3494900': [0, None, None], 'GSM3494901': [0, None, None], 'GSM3494902': [0, None, None], 'GSM3494903': [0, None, None], 'GSM3494904': [0, None, None], 'GSM3494905': [0, None, None], 'GSM3494906': [0, None, None], 'GSM3494907': [0, None, None], 'GSM3494908': [0, None, None], 'GSM3494909': [0, None, None], 'GSM3494910': [0, None, None], 'GSM3494911': [0, None, None], 'GSM3494912': [0, None, None], 'GSM3494913': [0, None, None], 'GSM3494914': [0, None, None], 'GSM3494915': [0, None, None], 'GSM

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}
